In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import re 
import pandas as pd
from splinter.exceptions import ElementDoesNotExist
import requests
import platform


In [2]:
system = platform.system()
print(system)
if system == 'Darwin':
    cd = 'chromedriver'
elif system == 'Windows':
    cd = 'chromedrive.exe'


Darwin


In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
def browser_init(url):
    
    executable_path = {'executable_path': cd}
    browser = Browser('chrome', **executable_path, headless=False)
    
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    browser.quit() 
    return soup

In [5]:
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
soup = browser_init(url)
title_div = soup.find('div', class_='content_title').text
p_div = soup.find('div' , class_ = 'article_teaser_body').text.strip()
print(f'{title_div}\n{p_div}')

Robotic Toolkit Added to NASA's Mars 2020 Rover
The bit carousel, which lies at the heart of the rover's Sample Caching System, is now aboard NASA's newest rover.


# PL Mars Space Images - Featured Image

In [ ]:
base_url = "https://www.jpl.nasa.gov"
soup = browser_init(base_url + "/spaceimages/?search=&category=Mars")

In [ ]:
featured_image = soup.find(class_ = 'carousel_item')
featured_image_a = featured_image.find('a')
featured_image_link =featured_image_a.get('data-fancybox-href') 
featured_image_link = base_url+featured_image_link
print(f'Link for the featured image(Mars) \n{featured_image_link}')

# Mars Weather

In [ ]:
twitter_base_url = 'https://twitter.com/marswxreport?lang=en'
soup = browser_init(twitter_base_url)
mars_weather_tweets = soup.find_all('p',class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')
for tweet in mars_weather_tweets:
    weather_tweet = tweet.text
    if(weather_tweet.startswith('InSight sol')):
        mars_weather = weather_tweet
        break
print(mars_weather)

# Mars Facts

In [ ]:
mars_facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(mars_facts_url)
planet_facts = tables[1]

planet_facts
planet_facts

In [ ]:
#converting to HTML 
facts_html_string = planet_facts.to_html(justify="left",index=False,header=False)
facts_html_string

# Mars Hemispheres

In [ ]:
def get_image_info(soup):
    # Get image URL
    image_info = soup.find(class_="downloads")
    image_info_link = image_info.find("li").find("a")
    img_url = image_info_link.get("href")

    # Get hemisphere title
    title = soup.find(class_="content").find(class_="title").text
    
    # Create dict object
    mars_hemi_dict = {}
    mars_hemi_dict["title"] = title
    mars_hemi_dict["img_url"] = img_url
    
    return mars_hemi_dict

In [ ]:
base_url = 'https://astrogeology.usgs.gov'
mars_hemi_url = base_url + '/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
soup = browser_init(mars_hemi_url)
links = []
parents = soup.select("#results-accordian")    
for parent in parents :
    
    for x in range(0,8, 2):
        a = parent.select('a',class_ = 'itemLink product-item')[x]
        links.append(base_url + a.get("href"))


In [ ]:
mars_hemi_info = []
for link in links:
    soup = browser_init(link)
    mars_hemi_info.append(get_image_info(soup))
    
print(mars_hemi_info)